In [7]:
import gzip,os,tarfile,sys
sys.path.append(os.pardir+'/src')
from settings import *
from boto3.session import Session
import datetime
import traceback
import logging
import pandas as pd
from pandas import DataFrame
import xml.etree.ElementTree as et

In [36]:
# S3から記事データをダウンロードする関数
def downloadFile(bucket, tag, target_day):
    # バケットから指定されたタグと日付に該当するオブジェクトを取得
    objects = bucket.objects.all().filter(Prefix=tag+target_day)

    for object in objects:
        # データを格納するパスを生成
        path = os.path.join(DATA_DIR,tag+target_day)
        # ダウンロードを実施
        bucket.download_file(object.key, path)

# startで指定された日付からspan日分のファイル名配列を生成
def makeDateList(start, span):
    dateList = []

    for i in range(int(span)):
        dateList.append('EID42168_' + start.strftime("%Y%m%d") + '.xml.gz')
        start = start + datetime.timedelta(days=1)

    return dateList

In [ ]:
start_time = datetime.datetime.now()

# コマンドライン引数からダウンロードを開始する日付と範囲を取得
start_date = '20150101'
span = '365'
start = datetime.datetime.strptime(start_date, '%Y%m%d')

# S3へ接続
session = Session(aws_access_key_id=AWS_ACCESS_KEY_ID,aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
s3 = session.resource('s3')
bucket = s3.Bucket(BUCKET_NAME)

# tagと日付リストを設定
tag = "EID42168_"
dateList = makeDateList(start, span)

for date in dateList:
    time = datetime.datetime.now()
    date = date[9:]
    try:
        downloadFile(bucket, tag, date)
        print(date + ' was done ' + str(datetime.datetime.now()-time))
    except Exception as e:
        print('error! ' + date)
        print(logging.error(traceback.format_exc()))

print('it taked ' + str(datetime.datetime.now() - start_time))

In [ ]:
# xml.gzファイルを解凍、xmlの構造を解析しCSVファイルを作成する関数
def convertToCSV(file_name):
    start = datetime.datetime.now()

    # 引数として渡されたtar.gzファイルを解凍し、オープン
    f = gzip.open(os.path.join(DATA_DIR,'originalData',file_name), 'r')

    # CSVファイルのカラムに対応する配列を初期化
    ids = []
    headlines = []
    timeofarrivals = []
    bodys = []
    #langs = []

    # xmlを解析し、rootを取得
    tree = et.parse(f)
    elem = tree.getroot()
    # エラーとなった記事をカウントする変数を初期化
    fail_cnt = 0

    # 解析したxmlから、記事単位で要素を取得
    contents = elem.getiterator('ContentT')

    for content in contents:
        try:
            # 記事の言語情報を取得し、日本語か英語の記事であれば以降の処理を実施
            lang = content.find(".//LanguageString").text
            #if lang == 'JAPANESE' or lang == 'ENGLISH':
            if lang == 'JAPANESE':
                if content.find(".//Body").text is not ' ':
                    # 言語、ID、タイトル、タイムスタンプを配列に格納
                    #langs.append(lang)
                    ids.append(content.find(".//Id/SUID").text)
                    headlines.append(content.find(".//Headline").text)
                    timeofarrivals.append(content.find(".//TimeOfArrival").text)
                    bodys.append(content.find(".//Body").text)
        except:
            # 読み取りに失敗した場合はカウント
            fail_cnt += 1
            #traceback.print_exc()
    
    print('fail_cnt:', fail_cnt)
    
    # カラムに対応する配列を用いてDataFrameを作成
    df = DataFrame({"Id":ids})
    df['Headline']=headlines
    df['Body']=bodys
    df['TimeOfArrival']=timeofarrivals
    df.drop_duplicates()
    # DataFrameをCSVファイルとして保存
    df.to_csv(os.path.join(DATA_DIR,'csvData1',file_name.replace(".xml.gz",".csv")),encoding='utf8',index=False)
    f.close()

    print('file_name:' + file_name + ' time:' + str(datetime.datetime.now()-start) + ' record_count:' + str(len(df)) + ' fail_count:' + str(fail_cnt))

In [10]:
#　全ファイル取得
files = os.listdir(DATA_DIR+'/originalData')

In [ ]:
# dataフォルダ配下の全ファイルを取得
for i in range(0,len(files)):
    convertToCSV(files[i])

In [15]:
# data解析　実験
from janome.tokenizer import Tokenizer
file_name = files[1]
hoge = pd.read_csv(os.path.join(DATA_DIR,'csvData1',file_name.replace(".xml.gz",".csv")))
headlines = hoge['Headline']

In [83]:
# 使い方リマインド
t = Tokenizer()
#tokens = t.tokenize(headlines[0])
tokens = t.tokenize('安倍晋三首相と、麻生太郎副総理兼財務相が、新たな「密約」を結んだという情報が飛び込んできた。中島悠太郎とドナルド・トランプは眠い')
for token in tokens:
    #print(token)
    #if (token.part_of_speech.split(',')[0]=='名詞'):
    if (token.part_of_speech.split(',')[2]=='人名'):
        print(token)
        #and token.part_of_speech.split(',')[1]=='固有名詞'):
        

安倍	名詞,固有名詞,人名,姓,*,*,安倍,アベ,アベ
晋	名詞,固有名詞,人名,名,*,*,晋,ススム,ススム
麻生	名詞,固有名詞,人名,姓,*,*,麻生,アソウ,アソー
太郎	名詞,固有名詞,人名,名,*,*,太郎,タロウ,タロー
中島	名詞,固有名詞,人名,姓,*,*,中島,ナカジマ,ナカジマ
悠太	名詞,固有名詞,人名,名,*,*,悠太,ユウタ,ユータ
ドナルド	名詞,固有名詞,人名,名,*,*,ドナルド,ドナルド,ドナルド


In [65]:
file_name = files[1]
hoge = pd.read_csv(os.path.join(DATA_DIR,'csvData1',file_name.replace(".xml.gz",".csv"))).drop(['Id', 'Body', 'TimeOfArrival'],axis=1)

In [76]:
hoge.shape[0]

927

In [66]:
hoge.head()

,Headline
0,ＥＣＢのギリシャの銀行への緊急支援拡大にドイツが反対－ロイター
1,ＥＣＢのギリシャの銀行への緊急支援拡大にドイツが反対－ロイター
2,*日経平均始値は0.6％高の18103円98銭、ＴＯＰＩＸ0.8％高の1473.84
3,日本郵政グループは豪トール買収で合意－１株9.04豪ドルで
4,*ブリヂストン株が買い気配


In [79]:
t = Tokenizer()
for i in range(0,hoge.shape[0]):
    tokens = t.tokenize(hoge['Headline'][i])
    for token in tokens:
        #print(token)
        #if (token.part_of_speech.split(',')[0]=='名詞'):
        if (token.part_of_speech.split(',')[2]=='人名'):
            print(token)
            #and token.part_of_speech.split(',')[1]=='固有名詞'):

始	名詞,固有名詞,人名,名,*,*,始,ハジメ,ハジメ
トール	名詞,固有名詞,人名,姓,*,*,トール,トール,トール
モルガン	名詞,固有名詞,人名,姓,*,*,モルガン,モルガン,モルガン
モルガン	名詞,固有名詞,人名,姓,*,*,モルガン,モルガン,モルガン
小林	名詞,固有名詞,人名,姓,*,*,小林,コバヤシ,コバヤシ
モルガン	名詞,固有名詞,人名,姓,*,*,モルガン,モルガン,モルガン
モルガン	名詞,固有名詞,人名,姓,*,*,モルガン,モルガン,モルガン
モルガン	名詞,固有名詞,人名,姓,*,*,モルガン,モルガン,モルガン
黒田	名詞,固有名詞,人名,姓,*,*,黒田,クロダ,クロダ
モルガン	名詞,固有名詞,人名,姓,*,*,モルガン,モルガン,モルガン
黒田	名詞,固有名詞,人名,姓,*,*,黒田,クロダ,クロダ
上田	名詞,固有名詞,人名,姓,*,*,上田,ウエダ,ウエダ
モルガン	名詞,固有名詞,人名,姓,*,*,モルガン,モルガン,モルガン
モルガン	名詞,固有名詞,人名,姓,*,*,モルガン,モルガン,モルガン
上田	名詞,固有名詞,人名,姓,*,*,上田,ウエダ,ウエダ
黒田	名詞,固有名詞,人名,姓,*,*,黒田,クロダ,クロダ
モルガン	名詞,固有名詞,人名,姓,*,*,モルガン,モルガン,モルガン
モルガン	名詞,固有名詞,人名,姓,*,*,モルガン,モルガン,モルガン
上田	名詞,固有名詞,人名,姓,*,*,上田,ウエダ,ウエダ
モルガン	名詞,固有名詞,人名,姓,*,*,モルガン,モルガン,モルガン
モルガン	名詞,固有名詞,人名,姓,*,*,モルガン,モルガン,モルガン
上田	名詞,固有名詞,人名,姓,*,*,上田,ウエダ,ウエダ
黒田	名詞,固有名詞,人名,姓,*,*,黒田,クロダ,クロダ
野村	名詞,固有名詞,人名,姓,*,*,野村,ノムラ,ノムラ
堀場	名詞,固有名詞,人名,姓,*,*,堀場,ホリバ,ホリバ
野村	名詞,固有名詞,人名,姓,*,*,野村,ノムラ,ノムラ
モルガン	名詞,固有名詞,人名,姓,*,*,モルガン,モルガン,モルガン
モルガン	名詞,固有名詞,人名,姓,*,*,モルガン,モルガン,モルガン
上田	名詞,固有名詞,人名,姓,*,*,上田,ウエダ,ウエダ
モルガン	名詞,固有名詞,人名

In [92]:
type(hoge.stack().tolist())

list

In [104]:
from sklearn.feature_extraction.text import CountVectorizer
# just for measuring time
import time
start = time.time()

# need to recieve unicode text, this is callable for TfidfVectorizer
# need to recieve unicode text
def myTokenizer(text):
    #TARGET_CATEGORY = ["名詞", "動詞",  "形容詞", "副詞", "連体詞", "助動詞"]
    #wordsIn=[]
    #t = Tokenizer()
    #tokens = t.tokenize(text)
    #for token in tokens:
    #    tokenCategory = token.part_of_speech.split(',')[0]
    #    tokenBasic = token.base_form
    #    if  (tokenCategory=='名詞' and token.part_of_speech.split(',')[1]=='固有名詞'):
    #        wordsIn.append(token.surface)
    #    elif tokenCategory in TARGET_CATEGORY:
    #        if tokenBasic != '*':                               #if basic form can be defined
    #            wordsIn.append(tokenBasic)
    wordsIn=[]
    t = Tokenizer()
    tokens = t.tokenize(text)
    for token in tokens:
        #print(token)
        #if (token.part_of_speech.split(',')[0]=='名詞'):
        if (token.part_of_speech.split(',')[2]=='人名'):
            wordsIn.append(token.surface)
    return wordsIn


#vectorizer = CountVectorizer(ngram_range=(1, 2),tokenizer=myTokenizer)     
#tfidf_weighted_matrix = vectorizer.fit_transform(tweetsProcessed)
vectorizer = CountVectorizer(ngram_range=(1, 2),tokenizer=myTokenizer,min_df=2).fit(hoge.stack().tolist()) # stop_words = ''
bow = vectorizer.transform(hoge.stack().tolist())

# for time
elapsed_time = time.time() - start
print(elapsed_time)
print("bag_of_words with df as 2: {}\n".format(repr(bow)))

148.65454196929932


In [119]:
import numpy as np
max_value = bow.max(axis=0).toarray().ravel()
sorted_by_num = max_value.argsort()
feature_names = np.array(vectorizer.get_feature_names())
print("Features with highest tfidf: \n{}\n".format(
      feature_names[sorted_by_num[-30:]]))

Features with highest tfidf: 
['アイン' '秋野 氏' '秋野' '甘利 氏' '甘利' '氏' '李' '木内' '憲一郎 氏' '憲一郎' '小林' '安倍' '堀場 小林'
 '堀場' '吉田 憲一郎' '吉田' '加藤' '住友 住友' '上田' '三村' 'モルガン 氏' 'モルガン' 'パウエル' 'トール'
 'トム' 'シラー' 'アイン 氏' '野村' '黒田' '住友']



In [111]:
print("First 30 features:\n{}".format(vectorizer.vocabulary_))?

First 30 features:
{'トール': 4, 'モルガン': 6, '小林': 18, '黒田': 29, '上田': 9, '野村': 28, '堀場': 15, '堀場 小林': 16, '安倍': 17, '三村': 8, '木内': 21, '吉田': 13, '憲一郎': 19, '氏': 23, '吉田 憲一郎': 14, '憲一郎 氏': 20, '加藤': 12, 'モルガン 氏': 7, '住友': 10, '李': 22, '甘利': 24, '甘利 氏': 25, 'アイン': 0, 'アイン 氏': 1, 'トム': 3, 'パウエル': 5, '住友 住友': 11, 'シラー': 2, '秋野': 26, '秋野 氏': 27}


In [114]:
freqs = [(word, bow.getcol(idx).sum()) for word, idx in vectorizer.vocabulary_.items()]
#sort from largest to smallest
print (sorted (freqs, key = lambda x: -x[1]))

[('モルガン', 47), ('黒田', 32), ('氏', 29), ('上田', 21), ('住友', 20), ('トール', 18), ('アイン', 9), ('アイン 氏', 9), ('住友 住友', 9), ('野村', 7), ('堀場', 7), ('シラー', 5), ('小林', 4), ('安倍', 4), ('木内', 4), ('堀場 小林', 3), ('三村', 3), ('甘利', 3), ('甘利 氏', 3), ('吉田', 2), ('憲一郎', 2), ('吉田 憲一郎', 2), ('憲一郎 氏', 2), ('加藤', 2), ('モルガン 氏', 2), ('李', 2), ('トム', 2), ('パウエル', 2), ('秋野', 2), ('秋野 氏', 2)]


In [32]:
# https://labs.goo.ne.jp/api/jp/named-entity-extraction/
# うまくいかないようであれば

In [ ]:
# 図示にあたって

In [38]:
start = datetime.datetime.strptime('20140101', '%Y%m%d')

In [41]:
a = makeDateList(start, '7')
type(a)

list

In [46]:
a = datetime.datetime.strptime('20140201', '%Y%m%d')
b = datetime.datetime.strptime('20140301', '%Y%m%d')

In [49]:
a<b

True

In [ ]:
# date:headlineのdictを受け取る
# flagは、月=(month)か年(=year)か
def shapeData(data, flag):
    res = pd.DataFrame()
    if (flag=='month'):
        
    elif(flag=='year'):
        
    else:
        raise Exception
    
    return res